In [ ]:
import pickle
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Initialize WebDriver
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open Instagram login page
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)

# Handle consent buttons
consent_button_xpaths = [
    "//button[contains(text(),'Allow all cookies')]",
    "//button[contains(text(),'Accept')]",
    "//button[contains(text(),'Consent')]",
    "//button[contains(text(),'Agree')]"
]

for xpath in consent_button_xpaths:
    try:
        consent_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        consent_button.click()
        print(f"Clicked consent button: {xpath}")
        time.sleep(2)
        break
    except:
        continue

# Login
username = ""
password = ""

username_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
password_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "password"))
)

username_input.send_keys(username)
password_input.send_keys(password)

login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
)
login_button.click()

# Wait for login to complete
time.sleep(5)

# Save cookies
cookies = driver.get_cookies()
with open("cookies.pkl", "wb") as file:
    pickle.dump(cookies, file)
print("Cookies saved!")

# Quit WebDriver
driver.quit()


Clicked consent button: //button[contains(text(),'Allow all cookies')]
Cookies saved!


In [ ]:
import pickle
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Instagram pages to scrape
instagram_pages = [
    "https://www.instagram.com/valorgi/",
    "https://www.instagram.com/hotfreestyle/"
]

# Initialize WebDriver
chrome_options = Options()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open Instagram to apply cookies
driver.get("https://www.instagram.com/")
time.sleep(5)

# Load cookies
try:
    with open("cookies.pkl", "rb") as file:
        cookies = pickle.load(file)
    for cookie in cookies:
        driver.add_cookie(cookie)
    print("Cookies loaded successfully!")
except FileNotFoundError:
    print("No saved cookies found! Please log in first.")

# Refresh the page to apply cookies
driver.refresh()
time.sleep(5)

# Function to extract posts
def extract_posts(soup):
    posts = []
    post_containers = soup.find_all("div", {"class": "x1lliihq x1n2onr6 xh8yej3 x4gyw5p x11i5rnm x1ntc13c x9i3mqj x2pgyrj"})
    
    for container in post_containers[:5]:  # Limit to first 5 posts
        post_data = {}
        
        # Extract post URL
        try:
            post_url = "https://www.instagram.com" + container.find("a")["href"]
            post_data["post_url"] = post_url
        except:
            post_data["post_url"] = "No URL"
        
        # Extract caption from the <img> tag inside the container
        try:
            img_tag = container.find("img", {"class": "x5yr21d xu96u03 x10l6tqk x13vifvy x87ps6o xh8yej3"})
            caption = img_tag["alt"]
            post_data["caption"] = caption
        except:
            post_data["caption"] = "No caption"
        
        posts.append(post_data)
    
    return posts

# Scrape content from each Instagram page
output = {}
for page in instagram_pages:
    print(f"Scraping: {page}")
    driver.get(page)
    time.sleep(5)

    # Wait for profile page to load
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
    except:
        print(f"Failed to load {page}")
        continue

    # Get page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract posts
    posts = extract_posts(soup)
    output[page] = posts

# Close WebDriver
driver.quit()

# Print JSON output with ensure_ascii=False
print(json.dumps(output, indent=4, ensure_ascii=False))

Cookies loaded successfully!
Scraping: https://www.instagram.com/valorgi/
Scraping: https://www.instagram.com/hotfreestyle/
{
    "https://www.instagram.com/valorgi/": [
        {
            "post_url": "https://www.instagram.com/valorgi/reel/DGq3X4AOboe/",
            "caption": "This person transformed a desert into a city of the future in just 50 years.\n⠀\nWe help people improve their lives through business and educational content.\n⠀\nJoin us on our journey 🔸 @Valorgi\n⠀\n⠀\nDM for credit or removal request (no copyright intended for footage video and music) © All rights and credits reserved to the respective owner(s)"
        },
        {
            "post_url": "https://www.instagram.com/valorgi/p/DGqkmTYO4Gh/",
            "caption": "Photo by Valorgi™ | Business | Wisdom on March 01, 2025. May be a meme of 3 people, people standing, people kissing, poster and text that says '滿 MAos VALORGI WHEN YOU TWO GET INTO A FIGHT, REMEMBER, IT'S YOU AND HER VS THE PROBLEM. NOT YOU VS HE

In [3]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from io import BytesIO

# Set up Chrome WebDriver
def get_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Run in background
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Function to perform reverse image search
def reverse_image_search(image_path, source="google"):
    driver = get_driver()
    
    if source == "google":
        driver.get("https://www.google.com/imghp?hl=en")
        time.sleep(2)

        # Click on camera icon (Google)
        camera_button = driver.find_element(By.CLASS_NAME, "nDcEnd")
        camera_button.click()
        time.sleep(2)

        # Upload image
        upload_input = driver.find_element(By.NAME, "encoded_image")
        upload_input.send_keys(os.path.abspath(image_path))
        time.sleep(5)  # Wait for search results to load

    elif source == "bing":
        driver.get("https://www.bing.com/visualsearch")
        time.sleep(2)

        # Upload image (Bing)
        upload_button = driver.find_element(By.ID, "sb_img_input")
        upload_button.send_keys(os.path.abspath(image_path))
        time.sleep(5)

    else:
        print("❌ Invalid source! Use 'google' or 'bing'.")
        return []

    # Scrape image URLs from search results
    img_elements = driver.find_elements(By.TAG_NAME, "img")
    img_urls = [img.get_attribute("src") for img in img_elements if img.get_attribute("src")]

    driver.quit()
    
    return img_urls[:5]  # Return top 5 image URLs

# Function to download images
def download_images(img_urls, folder="search_results"):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i, url in enumerate(img_urls):
        try:
            response = requests.get(url)
            image = Image.open(BytesIO(response.content))
            image.save(f"{folder}/result_{i+1}.jpg")
            print(f"✅ Saved: result_{i+1}.jpg")
        except Exception as e:
            print(f"❌ Failed to save image {i+1}: {e}")

# Run the reverse image search
image_path = "https://m.media-amazon.com/images/M/MV5BNzRiOTRiNmEtMWQxOC00NjY2LTk0NGYtYjA1OGFjOGY4OTdkXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg"  # Change this to your image file
search_source = "bing"  # Change to "bing" if needed

image_urls = reverse_image_search(image_path, source=search_source)
download_images(image_urls)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="sb_img_input"]"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x01120B43+25139]
	(No symbol) [0x010B13F4]
	(No symbol) [0x00F904E3]
	(No symbol) [0x00FD83D7]
	(No symbol) [0x00FD872B]
	(No symbol) [0x01021002]
	(No symbol) [0x00FFD014]
	(No symbol) [0x0101E778]
	(No symbol) [0x00FFCDC6]
	(No symbol) [0x00FCBDE9]
	(No symbol) [0x00FCD124]
	GetHandleVerifier [0x01424373+3185251]
	GetHandleVerifier [0x0144291A+3309578]
	GetHandleVerifier [0x0143CF42+3286578]
	GetHandleVerifier [0x011B7AE0+643536]
	(No symbol) [0x010BA20D]
	(No symbol) [0x010B70B8]
	(No symbol) [0x010B7257]
	(No symbol) [0x010A9E00]
	BaseThreadInitThunk [0x7599FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776282AE+286]
	RtlGetAppContain

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
from urllib.parse import urljoin

def setup_driver():
    """Set up and return a configured Chrome WebDriver"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode, comment this out if you want to see the browser
    chrome_options.add_argument("--window-size=1920,1080")
    
    # Replace with your chromedriver path if needed
    # service = Service('/path/to/chromedriver')
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def select_operational_reports(driver, url):
    """Navigate to the page and select operational reports"""
    driver.get(url)
    
    # Wait for dropdown and select 'Operational Report'
    wait = WebDriverWait(driver, 10)
    dropdown = wait.until(EC.presence_of_element_located((By.ID, 'contentbody_bodycontent_maincolumn_ddlDocTpe')))
    Select(dropdown).select_by_visible_text('Operational Report')
    
    # Click the 'Get Documents' button
    button = wait.until(EC.element_to_be_clickable((By.ID, 'contentbody_bodycontent_maincolumn_btngetdocs')))
    button.click()
    
    # Allow time for the page to load
    time.sleep(4)

def extract_pdf_links(driver, base_url):
    """Extract all PDF links from the current page"""
    pdf_data = []
    
    # Get all rows except the header and pagination rows
    rows = driver.find_elements(By.CSS_SELECTOR, "tr.gvRow")
    
    for row in rows:
        # Extract information
        columns = row.find_elements(By.TAG_NAME, "td")
        if len(columns) >= 3:
            title = columns[0].text.strip()
            date = columns[1].text.strip()
            
            # Get the PDF link
            link_element = columns[2].find_element(By.TAG_NAME, "a")
            relative_link = link_element.get_attribute("href")
            
            # Make the link absolute
            absolute_link = urljoin(base_url, relative_link) if relative_link else None
            
            pdf_data.append({
                "Title": title,
                "Date": date,
                "PDF_Link": absolute_link
            })
    
    return pdf_data

def get_current_page_number(driver):
    """Get the current page number by finding the <span> element in pagination"""
    try:
        span_element = driver.find_element(By.CSS_SELECTOR, "tr.pagination-ys span")
        return int(span_element.text.strip())
    except:
        return None

def click_next_page_set(driver, current_page):
    """Click the correct '...' link to move to the next set of pages"""
    # Get all ellipsis links
    ellipsis_links = driver.find_elements(By.XPATH, "//tr[@class='pagination-ys']//a[text()='...']")
    
    if len(ellipsis_links) == 1:
        # Only one ellipsis, just click it
        ellipsis_links[0].click()
    elif len(ellipsis_links) >= 2:
        # Two ellipsis links - we need to click the right one
        # If we're at page 1-10, click the second ellipsis (to go forward)
        # If we're at page 11-20, click the second ellipsis (to go forward)
        # If we're at page 21-30, might need different logic
        
        # Find the pagination links to determine context
        pagination_links = driver.find_elements(By.CSS_SELECTOR, "tr.pagination-ys a")
        page_numbers = []
        
        for link in pagination_links:
            try:
                page_num = int(link.text.strip())
                page_numbers.append(page_num)
            except:
                pass
        
        if page_numbers:
            min_page = min(page_numbers)
            max_page = max(page_numbers)
            
            if min_page <= 10 and max_page <= 10:
                # We're in the first set (pages 1-10), click the forward ellipsis
                ellipsis_links[-1].click()  # Last ellipsis
            elif min_page >= 11 and max_page <= 20:
                # We're in the second set (pages 11-20), click the forward ellipsis
                ellipsis_links[-1].click()  # Last ellipsis
            else:
                # We're in a different set, click based on current page
                if current_page <= 10:
                    ellipsis_links[-1].click()  # Click the last ellipsis to go forward
                elif current_page <= 20:
                    ellipsis_links[-1].click()  # Click the last ellipsis to go forward
                else:
                    ellipsis_links[-1].click()  # Default to the last one
    
    time.sleep(6)  # Wait for the page to load

def navigate_pagination(driver, start_page, end_page):
    """Navigate through pagination from start_page to end_page"""
    all_pdf_data = []
    current_page = get_current_page_number(driver)
    
    # If we're not on the start_page, we need to navigate there
    if current_page != start_page:
        # Find and click the link for the start page
        pagination_links = driver.find_elements(By.CSS_SELECTOR, "tr.pagination-ys a")
        for link in pagination_links:
            if link.text.strip() == str(start_page):
                link.click()
                time.sleep(6)
                break
    
    # Now we should be on the start_page
    for page_num in range(start_page, end_page + 1):
        # Extract data from current page
        print(f"Scraping page {page_num}...")
        pdf_data = extract_pdf_links(driver, "https://nsong.org/")
        all_pdf_data.extend(pdf_data)
        
        # If not the last page, navigate to the next page
        if page_num < end_page:
            # Find and click the link for the next page
            pagination_links = driver.find_elements(By.CSS_SELECTOR, "tr.pagination-ys a")
            
            # Find the specific link for the next page
            next_page_found = False
            for link in pagination_links:
                if link.text.strip() == str(page_num + 1):
                    link.click()
                    next_page_found = True
                    time.sleep(5)
                    break
                    
            # If we couldn't find the next page link, we might need to go to the next set
            if not next_page_found and page_num % 10 == 0:
                print(f"Clicking ellipsis to move from page {page_num} to next set...")
                click_next_page_set(driver, page_num)
    
    return all_pdf_data

def scrape_all_pages(driver, max_pages=382):
    """Scrape all pages up to max_pages"""
    all_pdf_data = []
    page_sets = [(1, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60), (61, 70), (71, 80), (81, 90), (91, 100),
                 (101, 110), (111, 120), (121, 130), (131, 140), (141, 150), (151, 160), (161, 170), (171, 180), (181, 190), (191, 200),
                 (201, 210), (211, 220), (221, 230), (231, 240), (241, 250), (251, 260), (261, 270), (271, 280), (281, 290), (291, 300),
                 (301, 310), (311, 320), (321, 330), (331, 340), (341, 350), (351, 360), (361, 370), (371, 380), (381, 382)]
    
    for start, end in page_sets:
        current_page = get_current_page_number(driver)
        print(f"Current page: {current_page}, Target range: {start}-{end}")
        
        # If we need to move to a different set
        if current_page is not None and (current_page < start or current_page > end):
            # We need to navigate to the appropriate set
            if current_page < start:
                # Need to go forward
                while current_page < start:
                    click_next_page_set(driver, current_page)
                    current_page = get_current_page_number(driver)
            else:
                # Need to go backward
                while current_page > end:
                    # Click the first ellipsis to go back
                    ellipsis_links = driver.find_elements(By.XPATH, "//tr[@class='pagination-ys']//a[text()='...']")
                    if ellipsis_links:
                        ellipsis_links[0].click()  # First ellipsis to go back
                        time.sleep(4)
                        current_page = get_current_page_number(driver)
        
        # Now scrape this set
        pdf_data = navigate_pagination(driver, start, end)
        all_pdf_data.extend(pdf_data)
    
    return all_pdf_data

def main():
    url = 'https://nsong.org/Library.aspx'
    driver = setup_driver()
    
    try:
        # Step 1 & 2: Navigate to page and select operational reports
        select_operational_reports(driver, url)
        
        # Step 3: Scrape all pages up to 382
        all_pdf_data = scrape_all_pages(driver, max_pages=382)
        
        # Save the data to CSV
        df = pd.DataFrame(all_pdf_data)
        df.to_csv("nsong_operational_reports.csv", index=False)
        print(f"Scraping completed. Found {len(all_pdf_data)} PDF links.")
        
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

Current page: 1, Target range: 1-10
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Current page: 10, Target range: 11-20
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Current page: 20, Target range: 21-30
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Current page: 30, Target range: 31-40
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Current page: 40, Target range: 41-50
Scraping page 41...
S

In [ ]:
import aiohttp
import aiofiles
import asyncio
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Read the CSV file and extract PDF links
csv_file = 'nsong_operational_reports.csv'
df = pd.read_csv(csv_file)
pdf_urls = df['PDF_Link'].tolist()


In [15]:
print(pdf_urls)

['https://nsong.org/DownloadHandler.ashx?FileName=OP28022025F.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP27022025b.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP26022025b.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP25022025B.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP24022025b.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP23022025A.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP22022025b.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP21022025A.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP20022025.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP19022025B.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP18022025b.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP17022025B.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP16022025B.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP15022025B.pdf', 'https://nsong.org/DownloadHandler.ashx?FileName=OP14022025A.p

In [16]:
import aiohttp
import aiofiles
import asyncio
import pandas as pd
import zipfile
import os
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio

# Apply nest_asyncio to allow running asyncio event loop in a nested context
nest_asyncio.apply()

# Read the CSV file and extract PDF links
csv_file = 'nsong_operational_reports.csv'
df = pd.read_csv(csv_file)
pdf_urls = df['PDF_Link'].tolist()

# Print the length of the list of PDF URLs
print(f"Number of file links: {len(pdf_urls)}")

# Directory to save downloaded files
download_dir = 'downloads'

async def download_file(session, url, filename):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                async with aiofiles.open(filename, 'wb') as file:
                    content = await response.read()
                    await file.write(content)
                print(f"Downloaded {filename}")
            else:
                print(f"Failed to download {url}, status code: {response.status}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

async def main(pdf_urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, url in enumerate(pdf_urls):
            # Extract the file name and extension from the URL
            file_name = url.split('=')[-1]
            filename = f"{download_dir}/{file_name}"
            tasks.append(download_file(session, url, filename))

        # Limit concurrent downloads to 10 workers
        with ThreadPoolExecutor(max_workers=10) as executor:
            loop = asyncio.get_event_loop()
            await asyncio.gather(*tasks, return_exceptions=True)

    # Create a zip file of the downloaded files
    zip_filename = 'operational_reports.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, _, files in os.walk(download_dir):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), download_dir))
    print(f'All files downloaded and zipped into {zip_filename}')

# Create the download directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Create a new event loop and run the main function
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(main(pdf_urls))


Number of file links: 3811
Downloaded downloads/OP16122024B.pdf
Downloaded downloads/OP18112024B.pdf
Downloaded downloads/OP13012025.pdf
Downloaded downloads/OP02022025.pdf
Downloaded downloads/OP17112024B.pdf
Downloaded downloads/OP10122024B.pdf
Downloaded downloads/OP15012025B.pdf
Downloaded downloads/OP01012025.pdf
Downloaded downloads/op22012025B.pdf
Downloaded downloads/OP06122024b.pdf
Downloaded downloads/OP19012025B.pdf
Downloaded downloads/OP02012025B.pdf
Downloaded downloads/OP25122024B.pdf
Downloaded downloads/OP09012025B.pdf
Downloaded downloads/OP13112024B.pdf
Downloaded downloads/OP23112024B.pdf
Downloaded downloads/OP14022025A.pdf
Downloaded downloads/OP10012025B.pdf
Downloaded downloads/OP24022025b.pdf
Downloaded downloads/op27012025B.pdf
Downloaded downloads/OP30112024B.pdf
Downloaded downloads/OP19122024B.pdf
Downloaded downloads/OP15022025B.pdf
Downloaded downloads/OP12122024B.pdf
Downloaded downloads/OP21112024B.pdf
Downloaded downloads/OP12112024B.pdf
Downloaded dow